# Packages

In [1]:
#basic packages
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

#data pre-processing packages
from datetime import datetime


#results and analysis packages
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# data modelling & results
from yellowbrick.regressor import PredictionError, ResidualsPlot
from sklearn.model_selection import train_test_split

#NN
import tensorflow as tf

from tensorflow import keras
from keras.utils.vis_utils import plot_model
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

#feature importance
import shap


# Script

## Error computation

In [3]:
#defining the Root Mean Squared Error

def rmse(y_true, y_predicted):
    
    return np.sqrt(mean_squared_error(y_true, y_predicted))

In [4]:
#errors computation

def errors_computation(data):
    
    df=pd.DataFrame()
    #df.at['RMSE (as root mean)', 'Wind']= round(rmse(data['Target'], data['WS_pred']), 3)
    df.at['MAE (in avg)', 'Wind']= round(mae(data['Target'], data['WS_pred']), 3)
    df.at['MAPE (%)', 'Wind']= round(mape(data['Target'], data['WS_pred'])*100, 3)
    
    #df.at['RMSE (as root mean)', 'Power']= round(rmse(data['P'], data['P_pred']), 3)
    df.at['MAE (in avg)', 'Power']= round(mae(data['P'], data['P_pred']), 3)
    df.at['MAPE (%)', 'Power']= round(mape(data['P'], data['P_pred'])*100, 3)
    
    
    print('Wind RMSE: ', round(rmse(data['Target'], data['WS_pred']), 3), 'm/s as root mean')
    print('Wind MAE: ', round(mae(data['Target'], data['WS_pred']), 3), 'm/s in avg')
    print('Wind MAPE: ', round(mape(data['Target'], data['WS_pred'])*100, 3), '%')
    
    print('Power RMSE: ', round(rmse(data['P'], data['P_pred']), 3), 'kW as root mean')
    print('Power MAE: ', round(mae(data['P'], data['P_pred']), 3), 'kW in avg')
    print('Power MAPE: ', round(mape(data['P'], data['P_pred'])*100, 3), '%')
    
    return df

In [5]:
def error_plot(data, title):
    
    #title is expected to be an str
    #WS_pred and Target should be the variables names

    #plotting the reference
    plt.figure(figsize=(12,8))
    plt.plot([-1,17.5],[-1,17.5], 'green', linewidth=4, alpha=.12)
    plt.plot(data['WS_pred'], data['Target'], marker='o', ls='', label='Regression', markersize=5, alpha=.1)


    plt.legend()

    ax=plt.gca()
    ax.set(xlabel='y predicted', ylabel='y actual');
    ax.set_title(title)
    ax.set_ylim(ymin=4, ymax=17.5)
    ax.set_xlim(xmin=4, xmax=17.5)
    
    return print('')

In [6]:
def powercurve_computation(data, power_curve):
    
    from scipy import interpolate
    
    #this function computes the power at a observation given the information at a observation:
    # the WS (in m/s) at the wind turbine location and at the hub height (Target)
    # the power curve of the wind turbine in an xslx
    
    
    x=power_curve['Wind Speed [m/s]']
    y=power_curve['Warranted Power Curve [kW]']
    x_new=data['Target']
    
    f = interpolate.interp1d(x, y)
    #, kind='linear'
    data['P']=f(x_new)
    
    if 'WS_pred' in data.keys():
        x_new2=data['WS_pred']
        data['P_pred']=f(x_new2)
    
    print('power curve computation performed')
    
    return data

In [7]:
def control_power_computation (data_test, data_train, power_curve):
    
    results_test=pd.DataFrame()
    results_train=pd.DataFrame()
    
    
    results_test=powercurve_computation(data_test, power_curve)
    results_train=powercurve_computation(data_train, power_curve)

    return results_test, results_train

In [8]:
def compute_results(data_test, data_train, power_curve, plot_error):
    
    #this function computes and plots the results of a modelling:

    results_test, results_train=control_power_computation (data_test, data_train, power_curve)
    
    
    print('Modelling errors for training set:')
    errors_computation(results_train)
    print('')
    print('Modelling errors for test set:')
    errors_computation(results_test)
    
    if plot_error:
        print('')
        error_plot(results_test, 'Error plot for test set wind speed')

    print('')
    return print('Showing the results of the modelling: ')

## Data uploading

In [9]:
def uploading_csv(file_folder,file_name):
    
    #file folder required
    #file name required
    #file is expected to be in the data root: r'C:\Users\irgaa\Irma\Data'
    #this function uploads and formats csv/txt/xlsx datasets into DataFrame
    
    
    data_root=r'C:\Users\irgaa\Irma\Data'
    data_folder=str(file_folder)
    data_file=str(file_name)
    
    data_path=data_root+data_folder+data_file
    
    data1 = pd.read_csv(data_path)

    
    # We will save the WD_bin as the index
    
    return data1

In [10]:
#this function saves a data csv

def save (data, file_folder,file_name):
    
    #file folder required
    #file name required
    #file is expected to be saved in the data root: r'C:\Users\irgaa\Irma\Data'
    #this function saves a csv/txt/xlsx into Irma's folder
    #the saved file will keep the columns names but not the index
    
    data_root=r'C:\Users\irgaa\Irma\Data'
    data_folder=str(file_folder)
    data_file=str(file_name)
    
    data_path=data_root+data_folder+data_file
    
    data.to_csv (data_path, index = False, header=True)
    
    
    return print('file', data_file, 'saved in', data_folder, 'folder')

## Data selection

In [11]:
def data_selection(X_train, X_test, inputs):
    
    #this function returns the columns of the training and test sets in the inputs list
    
    X_train1 = pd.DataFrame()
    X_test1 = pd.DataFrame()
    
    
    X_train1 = X_train[inputs]
    X_test1 = X_test[inputs]

    
    return X_train1,X_test1

In [12]:
def data_drop(X_train, X_test, list_2drop):
    
    #this function returns the columns of the training and test sets in the inputs list

    X_train1 = X_train.drop(columns=list_2drop)
    X_test1 = X_test.drop(columns=list_2drop)

    
    
    return X_train1,X_test1

## Modelling

### Model building

In [13]:
def build_model (n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8],
                 activation='relu', optimizer='Adam', regularization=None, Leaky=False):
    
    #this function build model is created only for building the NN
    #it will always initialize the weights using the strategy 'He normal' unless activation function 'selu' is selected
    
    
    if activation!='relu':
        Leaky==False
    
    
    #we create a sequential model:
    model=keras.models.Sequential()
    
    #we add the input layer with n_neurons=n_features (shape of X_train)
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    #we add a hidden layer for each n_hidden with ReLU as activation function
    #this code considers the same n_neurons for each hidden layer
    for layer in range(n_hidden):
        
        if regularization=='l2':
            if activation=='relu':
                model.add(keras.layers.Dense(n_neurons, activation='relu', kernel_initializer='he_normal',
                                            kernel_regularizer=keras.regularizers.l2(0.01)))
            elif activation=='elu':
                model.add(keras.layers.Dense(n_neurons, activation='elu', kernel_initializer='he_normal',
                                            kernel_regularizer=keras.regularizers.l2(0.01)))
            elif activation=='selu':
                model.add(keras.layers.Dense(n_neurons, activation='selu', kernel_initializer='lecun_normal',
                                           kernel_regularizer=keras.regularizers.l2(0.01)))  
        elif regularization=='l1':
            if activation=='relu':
                model.add(keras.layers.Dense(n_neurons, activation='relu', kernel_initializer='he_normal',
                                             kernel_regularizer=keras.regularizers.l1(0.01)))
            elif activation=='elu':
                model.add(keras.layers.Dense(n_neurons, activation='elu', kernel_initializer='he_normal',
                                             kernel_regularizer=keras.regularizers.l1(0.01)))
            elif activation=='selu':
                model.add(keras.layers.Dense(n_neurons, activation='selu', kernel_initializer='lecun_normal',
                                            kernel_regularizer=keras.regularizers.l1(0.01))) 
        else:
            if activation=='relu':
                model.add(keras.layers.Dense(n_neurons, activation='relu', kernel_initializer='he_normal'))
            elif activation=='elu':
                model.add(keras.layers.Dense(n_neurons, activation='elu', kernel_initializer='he_normal'))
            elif activation=='selu':
                model.add(keras.layers.Dense(n_neurons, activation='selu', kernel_initializer='lecun_normal'))
        
        if Leaky:
            model.add(keras.layers.LeakyReLU(alpha=0.2))
    
    #dropout only considered in the last layer
    if regularization=='Dropout':
        model.add(keras.layers.Dropout(rate=0.2))
        
    #we add the output layer with one neuron (we only want to predict 1 target)
    model.add(keras.layers.Dense(1))
    
    #we choose our optimizer and build it:
    if optimizer=='SGD':
        optimizer=keras.optimizers.SGD(lr=learning_rate)
    elif optimizer=='Momentum':
        optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=0.9)
    elif optimizer=='Nesterov':
        optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    elif optimizer=='RMSprop':
        optimizer=keras.optimizers.RMSprop(lr=learning_rate, rho=0.9)
    elif optimizer=='Adam':
        optimizer=keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999)
    elif optimizer=='Nadam':
        optimizer=keras.optimizers.Nadam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
    
    #we compile our model with the selected optimizer and set the objective function loss='mse'
    model.compile(loss='mse', optimizer=optimizer)
    
    return model
    

In [14]:
def build_model_old (n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8],
                 activation='relu', optimizer='Adam', regularization=None, Leaky=False):
    
    #this function build model is created only for building the NN
    #it will always initialize the weights using the strategy 'He normal' unless activation function 'selu' is selected
    
    
    if activation!='relu':
        Leaky==False
    
    
    #we create a sequential model:
    model=keras.models.Sequential()
    
    #we add the input layer with n_neurons=n_features (shape of X_train)
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    
    #we add a hidden layer for each n_hidden with ReLU as activation function
    #this code considers the same n_neurons for each hidden layer
    for layer in range(n_hidden):
        if activation=='relu':
            model.add(keras.layers.Dense(n_neurons, activation='relu', kernel_initializer='he_normal'))
        elif activation=='elu':
            model.add(keras.layers.Dense(n_neurons, activation='elu', kernel_initializer='he_normal'))
        elif activation=='selu':
            model.add(keras.layers.Dense(n_neurons, activation='selu', kernel_initializer='lecun_normal'))
        if Leaky:
            model.add(keras.layers.LeakyReLU(alpha=0.2))
            
    if regularization=='Dropout':
        model.add(keras.layers.Dropout(rate=0.2))
        
    #we add the output layer with one neuron (we only want to predict 1 target)
    model.add(keras.layers.Dense(1))
    
    #we choose our optimizer and build it:
    if optimizer=='SGD':
        optimizer=keras.optimizers.SGD(lr=learning_rate)
    elif optimizer=='Momentum':
        optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=0.9)
    elif optimizer=='Nesterov':
        optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    elif optimizer=='RMSprop':
        optimizer=keras.optimizers.RMSprop(lr=learning_rate, rho=0.9)
    elif optimizer=='Adam':
        optimizer=keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999)
    elif optimizer=='Nadam':
        optimizer=keras.optimizers.Nadam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
    
    #we compile our model with the selected optimizer and set the objective function loss='mse'
    model.compile(loss='mse', optimizer=optimizer)
    
    return model
    

### Modelling NN

In [15]:
def modelling_NN (X, X_test, y, y_test, power_curve,  parameters, plot_error, plot):
    
    #creating the model
    
    n_hidden=parameters['n_hidden']
    n_neurons=parameters['n_neurons']
    learning_rate=parameters['learning_rate']
    input_shape=X.shape[1:]
    activation=parameters['activation']
    optimizer=parameters['optimizer']
    regularization=parameters['regularization']
    Leaky=parameters['Leaky']
    
    model =build_model(n_hidden, n_neurons, learning_rate, input_shape,
                 activation, optimizer, regularization, Leaky)
 
    
    #model fitting
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=42)
    

    if regularization=='Early Stopping':
        model.fit(X_train, y_train, epochs=100,
                      validation_data=(X_valid, y_valid),
                      callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    else:
        model.fit(X_train, y_train, epochs=100,
                      validation_data=(X_valid, y_valid))
    
    
#     if plot:
#         loss_epochs_plot (history)
        
    
    #model predicting
    y_pred_test=model.predict(X_test)
    y_pred_train=model.predict(X)
    y_pred_valid=model.predict(X_valid)
    
    rmse_valid=rmse(y_valid, y_pred_valid)
    
    test=pd.DataFrame(y_pred_test, columns = ['test'])
    train=pd.DataFrame(y_pred_train, columns = ['train'])
    
    

    #computing the results
    data_test = pd.DataFrame()
    data_train = pd.DataFrame()
    
    data_test['WS_pred']=test['test']
    data_test['Target']=y_test['Target']
    data_train['WS_pred']=train['train']
    data_train['Target']=y['Target']
    
    print('')
    print('RMSE for validation', rmse_valid)
    print('')
    
    
    compute_results(data_test, data_train, power_curve, plot_error)
    print('NN modelling performed')
    
    return model

In [16]:
def modelling_NN_ES (X, X_test, y, y_test, power_curve,  parameters, plot_error, plot):
    
    #creating the model
    
    n_hidden=parameters['n_hidden']
    n_neurons=parameters['n_neurons']
    learning_rate=parameters['learning_rate']
    input_shape=X.shape[1:]
    activation=parameters['activation']
    optimizer=parameters['optimizer']
    regularization=parameters['regularization']
    Leaky=parameters['Leaky']
    
    model =build_model(n_hidden, n_neurons, learning_rate, input_shape,
                 activation, optimizer, regularization, Leaky)
 
    
    #model fitting
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=42)
    

    model.fit(X_train, y_train, epochs=100,
                      validation_data=(X_valid, y_valid),
                      callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    
    
#     if plot:
#         loss_epochs_plot (history)
        
    
    #model predicting
    y_pred_test=model.predict(X_test)
    y_pred_train=model.predict(X)
    
    test=pd.DataFrame(y_pred_test, columns = ['test'])
    train=pd.DataFrame(y_pred_train, columns = ['train'])
    
    

    #computing the results
    data_test = pd.DataFrame()
    data_train = pd.DataFrame()
    
    data_test['WS_pred']=test['test']
    data_test['Target']=y_test['Target']
    data_train['WS_pred']=train['train']
    data_train['Target']=y['Target']
    
#     mse_test=model.evaluate(X_test, y_test)
#     rmse_test=np.sqrt(mse_test)
#     print('RMSE for test', rmse_test)
    
    
    compute_results(data_test, data_train, power_curve, plot_error)
    print('NN modelling performed')
    
    return model

### Random Search NN

In [17]:
def RandomSearch_NN(X, X_test, y, y_test, power_curve, param_distribs, plot_error):
    
    #counting the runing time
    start_time = time.time()
    
    
    #creating the model
    input_shape=X.shape[1:]
    param_distribs['input_shape']=input_shape
    keras_reg =keras.wrappers.scikit_learn.KerasRegressor(build_model, verbose=0)
    
    
    #Random Search CV
    rnd_search_cv=RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3)
    
    #model fitting
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=42)
    
    regularization=param_distribs['regularization']
    
    if regularization=='Early Stopping':
        rnd_search_cv.fit(X_train, y_train, epochs=100,
                      validation_data=(X_valid, y_valid),
                      callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    else:
        rnd_search_cv.fit(X_train, y_train, epochs=100,
                      validation_data=(X_valid, y_valid))
    
    
    #model predicting
    
    model=rnd_search_cv.best_estimator_.model
    y_pred_test=model.predict(X_test)
    y_pred_train=model.predict(X)
    
    test=pd.DataFrame(y_pred_test, columns = ['test'])
    train=pd.DataFrame(y_pred_train, columns = ['train'])
    
    print('')
    print('Best parameters :')
    print(rnd_search_cv.best_params_)
    print('Best score :')
    print(rnd_search_cv.best_score_)
    print('')
    print("--- %s minutes ---" % ((time.time() - start_time)/60))
    print('')

    #computing the results
    data_test = pd.DataFrame()
    data_train = pd.DataFrame()
    
    data_test['WS_pred']=test['test']
    data_test['Target']=y_test['Target']
    data_train['WS_pred']=train['train']
    data_train['Target']=y['Target']
    
    compute_results(data_test, data_train, power_curve, plot_error)
    print('RandomSearch_ NN performed')
    
    return model
    

In [18]:
def RandomSearch_NN_ES(X, X_test, y, y_test, power_curve, param_distribs, plot_error):
    
    #counting the runing time
    start_time = time.time()
    
    #creating the model
    input_shape=X.shape[1:]
    param_distribs['input_shape']=input_shape
    keras_reg =keras.wrappers.scikit_learn.KerasRegressor(build_model, verbose=0)
    
    #Random Search CV
    rnd_search_cv=RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3)
    
    #model fitting
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=42)
    
    rnd_search_cv.fit(X_train, y_train, epochs=100,
                      validation_data=(X_valid, y_valid),
                      callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    
    
    #model predicting
    
    model=rnd_search_cv.best_estimator_.model
    y_pred_test=model.predict(X_test)
    y_pred_train=model.predict(X)
    
    test=pd.DataFrame(y_pred_test, columns = ['test'])
    train=pd.DataFrame(y_pred_train, columns = ['train'])
    
    print('')
    print('Best parameters :')
    print(rnd_search_cv.best_params_)
    print('Best score :')
    print(rnd_search_cv.best_score_)
    print('')
    print("--- %s minutes ---" % ((time.time() - start_time)/60))
    print('')

    #computing the results
    data_test = pd.DataFrame()
    data_train = pd.DataFrame()
    
    data_test['WS_pred']=test['test']
    data_test['Target']=y_test['Target']
    data_train['WS_pred']=train['train']
    data_train['Target']=y['Target']
    
    compute_results(data_test, data_train, power_curve, plot_error)
    print('RandomSearch_ NN performed')
    
    return model

### Loss plot

In [19]:
def loss_epochs_plot (history):
    
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.show
    
    return print('Loss vs. epochs plot performed')
    
    

### Model Testing

In [20]:
def model_testing (X_train, X_test, y_train, y_test, power_curve, model, plot_error):

    

    y_pred_test=model.predict(X_test)
    y_pred_train=model.predict(X_train)
    
    test=pd.DataFrame(y_pred_test, columns = ['test'])
    train=pd.DataFrame(y_pred_train, columns = ['train'])


    data_test = pd.DataFrame()
    data_train = pd.DataFrame()
    
    data_test['WS_pred']=test['test']
    data_test['Target']=y_test['Target']
    data_train['WS_pred']=train['train']
    data_train['Target']=y_train['Target']
    
    plot_model(model, show_shapes=True, show_layer_names=True)
    
    
    compute_results(data_test, data_train, power_curve, plot_error)
    

    
    return print('NN results performed')

### Feature importance

In [21]:
def feature_importance (X_train, X_test, model):
    
    X_t, X_f, y_t, y_f = train_test_split(X_train,y_train, test_size=0.02, random_state=12)
    
    background = X_f.copy()
    
    explainer = shap.KernelExplainer(model.predict,background)
    shap_values = explainer.shap_values(X_test,nsamples=100)
    shap.summary_plot(shap_values, X_train, plot_type="bar")
    print('Feature importance through SHAP values performed')
    

    return shap_values

In [22]:
def transform_shap (shap_values, X_test):

    v=np.array(shap_values)
    d=v.reshape(X_test.shape)
    shap_v=pd.DataFrame(d)
    
    feature_list=X_test.columns
    shap_v.columns=feature_list
    shap_v=shap_v.abs()
    k=pd.DataFrame(shap_v.mean()).reset_index()
    k.columns=['variables','SHAP_abs']
    k.sort_values(by='variables')
    
    return k
    

# Data analysis

## Dataset1

In [23]:
#upload the dataset with file_folder, file_name
# data_up= uploading_csv('\Dataset1-Normal_Site','\data_comp14.csv')
X_train= uploading_csv('\General','\X_train1.csv')
X_test= uploading_csv('\General','\X_test14.csv')
y_train= uploading_csv('\General','\y_train1.csv')
y_test= uploading_csv('\General','\y_test14.csv')

X_test.keys()

Index(['T2', 'RH2', 'T1', 'RH1', 'PR1', 'AD1', 'PR2', 'AD2', 'Rain', 'WS1',
       'WS3', 'WS4', 'WD1', 'WD3', 'WD4', 'WSHor', 'WDHor', 'WSVer', 'WDVer',
       'TI', 'WSH', 'WD_bin', 'tod', 'WVeer'],
      dtype='object')

In [24]:
y_test.keys()

Index(['Target'], dtype='object')

In [25]:
X_test.head()

,T2,RH2,T1,RH1,PR1,AD1,PR2,AD2,Rain,WS1,...,WD4,WSHor,WDHor,WSVer,WDVer,TI,WSH,WD_bin,tod,WVeer
0,0.686743,0.162077,0.615702,0.254286,0.694987,0.391943,0.684273,0.320279,0.0,0.323555,...,0.626145,0.331420,0.605429,0.516197,0.518763,0.406958,0.212675,0.571429,0.636364,0.531706
1,0.535995,0.186494,0.495123,0.265058,0.447523,0.435208,0.448760,0.390702,0.0,0.470419,...,0.679629,0.477351,0.636622,0.414907,0.458028,0.504232,0.243045,0.571429,0.748252,0.489887
2,0.100213,0.993270,0.105457,0.989550,0.403515,0.804920,0.415173,0.792480,0.0,0.357223,...,0.746132,0.358311,0.776603,0.569422,0.516569,0.145157,0.520344,0.714286,0.174825,0.537451
3,0.127726,0.774422,0.110625,0.832482,0.599315,0.857346,0.609338,0.821287,0.0,0.419386,...,0.613695,0.425773,0.519552,0.481510,0.483901,0.354450,0.412821,0.428571,0.265734,0.459187
4,0.309523,0.703601,0.317238,0.730702,0.659098,0.667381,0.666267,0.661645,0.0,0.435593,...,0.522701,0.452193,0.400189,0.355088,0.395601,0.080932,0.352598,0.285714,0.895105,0.459064


In [26]:
PC= uploading_csv('\Dataset1-Normal_Site','\PC_1.15kgm-3.csv')

### RandomSearch

In [30]:
#pending

In [27]:
param_distribs={
    'n_hidden':[0, 1, 2, 3],
    'n_neurons': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate': [0.0001, 0.0003, 0.001, 0.003, 0.005, 0.01, 0.03],
    'activation':['relu', 'elu', 'selu'],
    'optimizer':['Adam', 'Nesterov', 'Momentum', 'Nadam', 'RMSProp'],
    'regularization':[None, 'Dropout', 'Early Stopping'],
    'Leaky':[True, False],
}

In [28]:
model= RandomSearch_NN(X_train, X_test, y_train, y_test, PC, param_distribs, plot_error=False)
#iter=20


Best parameters :
{'regularization': 'Early Stopping', 'optimizer': 'Momentum', 'n_neurons': 100, 'n_hidden': 1, 'learning_rate': 0.005, 'input_shape': 19, 'activation': 'relu', 'Leaky': True}
Best score :
-0.6488828857739767

--- 12.630461589495342 minutes ---

power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.774 m/s as root mean
Wind MAE:  0.575 m/s in avg
Wind MAPE:  7.982 %
Power RMSE:  266.681 kW as root mean
Power MAE:  152.839 kW in avg
Power MAPE:  7.492292220539012e+17 %

Modelling errors for test set:
Wind RMSE:  0.857 m/s as root mean
Wind MAE:  0.614 m/s in avg
Wind MAPE:  10.44 %
Power RMSE:  296.199 kW as root mean
Power MAE:  163.602 kW in avg
Power MAPE:  1.8436701770959404e+18 %

Showing the results of the modelling: 
RandomSearch_ NN performed


### Manual modelling

In [27]:
parameters={
    'n_hidden':3,
    'n_neurons': 80,
    'learning_rate':0.001,
    'activation':'elu',
    'optimizer':'Adam',
    'regularization':None,
    'Leaky':True
}

In [28]:
model = modelling_NN (X_train, X_test, y_train, y_test, PC, parameters, plot_error=False, plot=True)

Epoch 1/100
186/186 [==============================] - 1s 5ms/step - loss: 5.7273 - val_loss: 0.4670
Epoch 2/100
186/186 [==============================] - 1s 3ms/step - loss: 0.3802 - val_loss: 0.3325
Epoch 3/100
186/186 [==============================] - 0s 3ms/step - loss: 0.3359 - val_loss: 0.3050
Epoch 4/100
186/186 [==============================] - 0s 2ms/step - loss: 0.3324 - val_loss: 0.3062
Epoch 5/100
186/186 [==============================] - 1s 3ms/step - loss: 0.3249 - val_loss: 0.3092
Epoch 6/100
186/186 [==============================] - 1s 3ms/step - loss: 0.3081 - val_loss: 0.2799
Epoch 7/100
186/186 [==============================] - 1s 3ms/step - loss: 0.3102 - val_loss: 0.2857
Epoch 8/100
186/186 [==============================] - 1s 3ms/step - loss: 0.3236 - val_loss: 0.3113
Epoch 9/100
186/186 [==============================] - 1s 3ms/step - loss: 0.2924 - val_loss: 0.2778
Epoch 10/100
186/186 [==============================] - 1s 3ms/step - loss: 0.2903 - val_lo

Epoch 81/100
186/186 [==============================] - 2s 12ms/step - loss: 0.2293 - val_loss: 0.2528
Epoch 82/100
186/186 [==============================] - 1s 8ms/step - loss: 0.2141 - val_loss: 0.2351
Epoch 83/100
186/186 [==============================] - 3s 14ms/step - loss: 0.2283 - val_loss: 0.2268
Epoch 84/100
186/186 [==============================] - 2s 13ms/step - loss: 0.2135 - val_loss: 0.2252
Epoch 85/100
186/186 [==============================] - 2s 10ms/step - loss: 0.2129 - val_loss: 0.2202
Epoch 86/100
186/186 [==============================] - 2s 11ms/step - loss: 0.2150 - val_loss: 0.2291
Epoch 87/100
186/186 [==============================] - 3s 16ms/step - loss: 0.2095 - val_loss: 0.2711
Epoch 88/100
186/186 [==============================] - 2s 13ms/step - loss: 0.2115 - val_loss: 0.2581
Epoch 89/100
186/186 [==============================] - 3s 14ms/step - loss: 0.2104 - val_loss: 0.2323
Epoch 90/100
186/186 [==============================] - 2s 12ms/step - los

### Model saving

parameters={
    'n_hidden':3,
    'n_neurons': 80,
    'learning_rate':0.001,
    'activation':'elu',
    'optimizer':'Adam',
    'regularization':None,
    'Leaky':True
}

MAPE power: 8.67 %- WTG14

In [29]:
model.save('dataset1_ANN1.h5')

### Model testing

In [83]:
model=keras.models.load_model('dataset1_ANN1.h5')

In [31]:
#WTG15

In [32]:
#upload the dataset with file_folder, file_name
# data_up= uploading_csv('\Dataset1-Normal_Site','\data_comp14.csv')
X_train= uploading_csv('\General','\X_train1.csv')
X_test= uploading_csv('\General','\X_test15.csv')
y_train= uploading_csv('\General','\y_train1.csv')
y_test= uploading_csv('\General','\y_test15.csv')

X_test.keys()

Index(['T2', 'RH2', 'T1', 'RH1', 'PR1', 'AD1', 'PR2', 'AD2', 'Rain', 'WS1',
       'WS3', 'WS4', 'WD1', 'WD3', 'WD4', 'WSHor', 'WDHor', 'WSVer', 'WDVer',
       'TI', 'WSH', 'WD_bin', 'tod', 'WVeer'],
      dtype='object')

In [33]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.496 m/s as root mean
Wind MAE:  0.385 m/s in avg
Wind MAPE:  4.479 %
Power RMSE:  222.381 kW as root mean
Power MAE:  145.534 kW in avg
Power MAPE:  9.852 %

Modelling errors for test set:
Wind RMSE:  0.562 m/s as root mean
Wind MAE:  0.429 m/s in avg
Wind MAPE:  5.169 %
Power RMSE:  252.252 kW as root mean
Power MAE:  167.161 kW in avg
Power MAPE:  12.098 %

Showing the results of the modelling: 
NN results performed


## Dataset2

In [36]:
#DOING

In [38]:
#upload the dataset with file_folder, file_name
# data_up= uploading_csv('\Dataset1-Normal_Site','\data_comp14.csv')
X_train= uploading_csv('\General','\X_train2.csv')
X_test= uploading_csv('\General','\X_test11.csv')
y_train= uploading_csv('\General','\y_train2.csv')
y_test= uploading_csv('\General','\y_test11.csv')

X_test.keys()

Index(['WS1', 'WS3', 'WS4', 'WD1', 'WD4', 'WSHor', 'WSVer', 'WDHor', 'RH1',
       'Rain', 'WSH', 'WVeer', 'TI', 'WDVer', 'WD_bin', 'tod'],
      dtype='object')

In [39]:
y_test.keys()

Index(['Target'], dtype='object')

In [40]:
PC= uploading_csv('\Dataset2-Complex_Site','\PC_V117.csv')

In [41]:
X_train=X_train.drop(columns=['RH1'])
X_test=X_test.drop(columns=['RH1'])

### Random Search

In [42]:
#PENDING

In [55]:
param_distribs={
    'n_hidden':[0, 1, 2, 3],
    'n_neurons': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate': [0.0001, 0.0003, 0.001, 0.003, 0.005, 0.01, 0.03],
    'activation':['relu', 'elu', 'selu'],
    'optimizer':['Adam', 'Nesterov', 'Momentum', 'Nadam'],
    'regularization':[None, 'Dropout', 'Early Stopping'],
    'Leaky':[True, False],
}

In [56]:
model= RandomSearch_NN(X_train, X_test, y_train, y_test, PC, param_distribs, plot_error=False)
#iter=20

One or more of the test scores are non-finite: [ -1.07578381  -0.39619265 -16.41513602  -0.49583897  -0.47011769
  -0.53511881  -0.6136086  -16.43231742  -1.04724256          nan
  -0.40382962 -86.82775116          nan  -0.5123837   -0.46735646
  -0.44322575  -0.3673082   -0.4475245   -7.31817595  -0.48747708]



Best parameters :
{'regularization': 'Dropout', 'optimizer': 'Nesterov', 'n_neurons': 100, 'n_hidden': 2, 'learning_rate': 0.003, 'input_shape': 16, 'activation': 'relu', 'Leaky': False}
Best score :
-0.36730819940567017

--- 9.923941997687022 minutes ---

power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.566 m/s as root mean
Wind MAE:  0.435 m/s in avg
Wind MAPE:  4.637 %
Power RMSE:  234.791 kW as root mean
Power MAE:  149.818 kW in avg
Power MAPE:  10.28 %

Modelling errors for test set:
Wind RMSE:  0.581 m/s as root mean
Wind MAE:  0.453 m/s in avg
Wind MAPE:  4.874 %
Power RMSE:  232.359 kW as root mean
Power MAE:  153.256 kW in avg
Power MAPE:  10.934 %

Showing the results of the modelling: 
RandomSearch_ NN performed


### Manual modelling

In [43]:
parameters={
    'n_hidden':3,
    'n_neurons': 80,
    'learning_rate':0.001,
    'activation':'elu',
    'optimizer':'Adam',
    'regularization':None,
    'Leaky':True
}

In [44]:
model = modelling_NN (X_train, X_test, y_train, y_test, PC, parameters, plot_error=False, plot=True)

Epoch 1/100
372/372 [==============================] - 1s 2ms/step - loss: 5.0400 - val_loss: 1.0080
Epoch 2/100
372/372 [==============================] - 1s 2ms/step - loss: 1.0384 - val_loss: 0.9347
Epoch 3/100
372/372 [==============================] - 1s 2ms/step - loss: 0.9743 - val_loss: 0.8765
Epoch 4/100
372/372 [==============================] - 1s 2ms/step - loss: 0.9246 - val_loss: 0.8647
Epoch 5/100
372/372 [==============================] - 1s 2ms/step - loss: 0.8987 - val_loss: 0.8081
Epoch 6/100
372/372 [==============================] - 1s 2ms/step - loss: 0.8768 - val_loss: 0.7801
Epoch 7/100
372/372 [==============================] - 1s 2ms/step - loss: 0.8506 - val_loss: 0.7550
Epoch 8/100
372/372 [==============================] - 1s 2ms/step - loss: 0.8273 - val_loss: 0.8234
Epoch 9/100
372/372 [==============================] - 1s 2ms/step - loss: 0.8378 - val_loss: 0.7470
Epoch 10/100
372/372 [==============================] - 1s 3ms/step - loss: 0.8034 - val_lo

372/372 [==============================] - 1s 2ms/step - loss: 0.5944 - val_loss: 0.6143
Epoch 82/100
372/372 [==============================] - 1s 2ms/step - loss: 0.5947 - val_loss: 0.5586
Epoch 83/100
372/372 [==============================] - 1s 3ms/step - loss: 0.5897 - val_loss: 0.5977
Epoch 84/100
372/372 [==============================] - 1s 4ms/step - loss: 0.5881 - val_loss: 0.5912
Epoch 85/100
372/372 [==============================] - 1s 3ms/step - loss: 0.5942 - val_loss: 0.7637
Epoch 86/100
372/372 [==============================] - 1s 3ms/step - loss: 0.5950 - val_loss: 0.6342
Epoch 87/100
372/372 [==============================] - 1s 3ms/step - loss: 0.6000 - val_loss: 0.5775
Epoch 88/100
372/372 [==============================] - 1s 2ms/step - loss: 0.5814 - val_loss: 0.6080
Epoch 89/100
372/372 [==============================] - 1s 2ms/step - loss: 0.5799 - val_loss: 0.5630
Epoch 90/100
372/372 [==============================] - 1s 2ms/step - loss: 0.5826 - val_loss: 

### Model saving

parameters={
    'n_hidden':3,
    'n_neurons': 80,
    'learning_rate':0.001,
    'activation':'elu',
    'optimizer':'Adam',
    'regularization':None,
    'Leaky':True
}

MAPE wind T11: 9.369%

In [45]:
model.save('dataset2_ANN1.h5')

### Model testing

In [46]:
model=keras.models.load_model('dataset2_ANN1.h5')

In [47]:
#T11

In [48]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.757 m/s as root mean
Wind MAE:  0.556 m/s in avg
Wind MAPE:  7.79 %
Power RMSE:  309.456 kW as root mean
Power MAE:  183.964 kW in avg
Power MAPE:  6.290893209686348e+17 %

Modelling errors for test set:
Wind RMSE:  0.866 m/s as root mean
Wind MAE:  0.614 m/s in avg
Wind MAPE:  9.369 %
Power RMSE:  328.203 kW as root mean
Power MAE:  175.158 kW in avg
Power MAPE:  1.1541772249653957e+18 %

Showing the results of the modelling: 
NN results performed


In [49]:
#T17

In [50]:
#upload the dataset with file_folder, file_name
# data_up= uploading_csv('\Dataset1-Normal_Site','\data_comp14.csv')
X_train= uploading_csv('\General','\X_train2.csv')
X_test= uploading_csv('\General','\X_test17.csv')
y_train= uploading_csv('\General','\y_train2.csv')
y_test= uploading_csv('\General','\y_test17.csv')

X_test.keys()

Index(['WS1', 'WS3', 'WS4', 'WD1', 'WD4', 'WSHor', 'WSVer', 'WDHor', 'RH1',
       'Rain', 'WSH', 'WVeer', 'TI', 'WDVer', 'WD_bin', 'tod'],
      dtype='object')

In [51]:
X_train=X_train.drop(columns=['RH1'])
X_test=X_test.drop(columns=['RH1'])

In [52]:
PC= uploading_csv('\Dataset2-Complex_Site','\PC_V112.csv')

In [53]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.757 m/s as root mean
Wind MAE:  0.556 m/s in avg
Wind MAPE:  7.79 %
Power RMSE:  305.568 kW as root mean
Power MAE:  187.497 kW in avg
Power MAPE:  6.128421798360227e+17 %

Modelling errors for test set:
Wind RMSE:  0.666 m/s as root mean
Wind MAE:  0.51 m/s in avg
Wind MAPE:  5.612 %
Power RMSE:  280.857 kW as root mean
Power MAE:  175.886 kW in avg
Power MAPE:  13.166 %

Showing the results of the modelling: 
NN results performed


In [54]:
# T22

In [55]:
#upload the dataset with file_folder, file_name
# data_up= uploading_csv('\Dataset1-Normal_Site','\data_comp14.csv')
X_train= uploading_csv('\General','\X_train2.csv')
X_test= uploading_csv('\General','\X_test22.csv')
y_train= uploading_csv('\General','\y_train2.csv')
y_test= uploading_csv('\General','\y_test22.csv')

X_test.keys()

Index(['WS1', 'WS3', 'WS4', 'WD1', 'WD4', 'WSHor', 'WSVer', 'WDHor', 'RH1',
       'Rain', 'WSH', 'WVeer', 'TI', 'WDVer', 'WD_bin', 'tod'],
      dtype='object')

In [56]:
X_train=X_train.drop(columns=['RH1'])
X_test=X_test.drop(columns=['RH1'])

In [57]:
PC= uploading_csv('\Dataset2-Complex_Site','\PC_V112.csv')

In [58]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.757 m/s as root mean
Wind MAE:  0.556 m/s in avg
Wind MAPE:  7.79 %
Power RMSE:  305.568 kW as root mean
Power MAE:  187.497 kW in avg
Power MAPE:  6.128421798360227e+17 %

Modelling errors for test set:
Wind RMSE:  0.794 m/s as root mean
Wind MAE:  0.604 m/s in avg
Wind MAPE:  7.993 %
Power RMSE:  349.184 kW as root mean
Power MAE:  239.061 kW in avg
Power MAPE:  33.547 %

Showing the results of the modelling: 
NN results performed


## Dataset3

In [24]:
#upload the dataset with file_folder, file_name
# data_up= uploading_csv('\Dataset1-Normal_Site','\data_comp14.csv')
X_train= uploading_csv('\General','\X_train3.csv')
X_test= uploading_csv('\General','\X_test18.csv')
y_train= uploading_csv('\General','\y_train3.csv')
y_test= uploading_csv('\General','\y_test18.csv')

X_test.keys()

Index(['WS1', 'WS3', 'WS4', 'WSHor', 'WDHor', 'WSVer', 'WDVer', 'T1', 'RH1',
       'T2', 'RH2', 'PR1', 'AD1', 'PR2', 'AD2', 'Rain', 'WD1', 'WD3', 'WD4',
       'TI', 'WSH', 'WD_bin', 'tod', 'WVeer'],
      dtype='object')

In [25]:
y_test.keys()

Index(['Target'], dtype='object')

In [26]:
PC= uploading_csv('\Dataset3-New_Site','\PC_V150.csv')

### Random Search

In [23]:
#pending

In [29]:
param_distribs={
    'n_hidden':[0, 1, 2, 3],
    'n_neurons': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate': [0.0001, 0.0003, 0.001, 0.003, 0.005, 0.01, 0.03],
    'activation':['relu', 'elu', 'selu'],
    'optimizer':['Adam', 'Nesterov', 'Momentum', 'Nadam'],
    'regularization':[None, 'Dropout', 'Early Stopping'],
    'Leaky':[True, False],
}

In [30]:
model= RandomSearch_NN(X_train, X_test, y_train, y_test, PC, param_distribs, plot_error=False)
#iter=20


Best parameters :
{'regularization': 'Dropout', 'optimizer': 'Nesterov', 'n_neurons': 80, 'n_hidden': 3, 'learning_rate': 0.001, 'input_shape': 17, 'activation': 'relu', 'Leaky': True}
Best score :
-0.6610205769538879

--- 9.078933183352152 minutes ---

power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.707 m/s as root mean
Wind MAE:  0.538 m/s in avg
Wind MAPE:  7.318 %
Power RMSE:  317.923 kW as root mean
Power MAE:  213.835 kW in avg
Power MAPE:  7.421315656754958e+16 %

Modelling errors for test set:
Wind RMSE:  0.741 m/s as root mean
Wind MAE:  0.559 m/s in avg
Wind MAPE:  7.467 %
Power RMSE:  348.899 kW as root mean
Power MAE:  227.326 kW in avg
Power MAPE:  24.052 %

Showing the results of the modelling: 
RandomSearch_ NN performed


In [31]:
param_distribs={
    'n_hidden':[0, 1, 2, 3],
    'n_neurons': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate': [0.0001, 0.0003, 0.001, 0.003, 0.005, 0.01, 0.03],
    'activation':['relu', 'elu', 'selu'],
    'optimizer':['Adam', 'Nesterov', 'Momentum', 'Nadam'],
    'regularization':[None, 'Dropout', 'Early Stopping'],
    'Leaky':[True, False],
}

In [32]:
model= RandomSearch_NN(X_train, X_test, y_train, y_test, PC, param_distribs, plot_error=False)
#iter=20

One or more of the test scores are non-finite: [-1.37643977 -1.69325157 -1.64425476 -1.00734305 -0.82106346 -0.83543595
 -0.89177605 -1.17182251 -0.69177105 -0.77259185 -1.05052233 -0.96071255
 -0.71196856 -0.72100133         nan -0.78692005 -0.82343479 -0.99675479
 -0.96730433 -1.00919571]



Best parameters :
{'regularization': 'Dropout', 'optimizer': 'Nesterov', 'n_neurons': 30, 'n_hidden': 2, 'learning_rate': 0.003, 'input_shape': 17, 'activation': 'relu', 'Leaky': True}
Best score :
-0.6917710502942404

--- 8.941617663701376 minutes ---

power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.844 m/s as root mean
Wind MAE:  0.646 m/s in avg
Wind MAPE:  8.645 %
Power RMSE:  379.044 kW as root mean
Power MAE:  251.963 kW in avg
Power MAPE:  7.349464790500515e+16 %

Modelling errors for test set:
Wind RMSE:  0.821 m/s as root mean
Wind MAE:  0.621 m/s in avg
Wind MAPE:  8.116 %
Power RMSE:  378.153 kW as root mean
Power MAE:  242.28 kW in avg
Power MAPE:  24.985 %

Showing the results of the modelling: 
RandomSearch_ NN performed


### Manual modelling

In [27]:
parameters={
    'n_hidden':3,
    'n_neurons': 80,
    'learning_rate':0.001,
    'activation':'elu',
    'optimizer':'Adam',
    'regularization':None,
    'Leaky':True
}

In [28]:
model = modelling_NN (X_train, X_test, y_train, y_test, PC, parameters, plot_error=False, plot=True)

Epoch 1/100
449/449 [==============================] - 1s 3ms/step - loss: 2.4088 - val_loss: 0.3557
Epoch 2/100
449/449 [==============================] - 1s 2ms/step - loss: 0.3397 - val_loss: 0.3201
Epoch 3/100
449/449 [==============================] - 1s 2ms/step - loss: 0.3256 - val_loss: 0.3466
Epoch 4/100
449/449 [==============================] - 1s 2ms/step - loss: 0.3294 - val_loss: 0.3078
Epoch 5/100
449/449 [==============================] - 1s 2ms/step - loss: 0.3063 - val_loss: 0.2974
Epoch 6/100
449/449 [==============================] - 1s 2ms/step - loss: 0.3123 - val_loss: 0.3854
Epoch 7/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2995 - val_loss: 0.2889
Epoch 8/100
449/449 [==============================] - 1s 2ms/step - loss: 0.3016 - val_loss: 0.2864
Epoch 9/100
449/449 [==============================] - 1s 2ms/step - loss: 0.3003 - val_loss: 0.2876
Epoch 10/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2912 - val_lo

449/449 [==============================] - 1s 2ms/step - loss: 0.2277 - val_loss: 0.2461
Epoch 82/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2229 - val_loss: 0.2530
Epoch 83/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2224 - val_loss: 0.2510
Epoch 84/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2240 - val_loss: 0.2474
Epoch 85/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2160 - val_loss: 0.2492
Epoch 86/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2230 - val_loss: 0.2740
Epoch 87/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2185 - val_loss: 0.2455
Epoch 88/100
449/449 [==============================] - 1s 2ms/step - loss: 0.2188 - val_loss: 0.2628
Epoch 89/100
449/449 [==============================] - 1s 3ms/step - loss: 0.2159 - val_loss: 0.2588
Epoch 90/100
449/449 [==============================] - 1s 3ms/step - loss: 0.2171 - val_loss: 

### Model saving

parameters={
    'n_hidden':3,
    'n_neurons': 80,
    'learning_rate':0.001,
    'activation':'elu',
    'optimizer':'Adam',
    'regularization':None,
    'Leaky':True
}

MAPE wind: 4.526% for WTG18

In [29]:
model.save('dataset3_ANN1.h5')

### Model testing

In [30]:
model=keras.models.load_model('dataset3_ANN1.h5')

### WTG18

In [31]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.457 m/s as root mean
Wind MAE:  0.338 m/s in avg
Wind MAPE:  4.256 %
Power RMSE:  259.989 kW as root mean
Power MAE:  168.729 kW in avg
Power MAPE:  10.304 %

Modelling errors for test set:
Wind RMSE:  0.489 m/s as root mean
Wind MAE:  0.349 m/s in avg
Wind MAPE:  4.526 %
Power RMSE:  268.909 kW as root mean
Power MAE:  176.203 kW in avg
Power MAPE:  21.752 %

Showing the results of the modelling: 
NN results performed


### WTG20

In [32]:
#upload the dataset with file_folder, file_name
X_train= uploading_csv('\General','\X_train3.csv')
X_test= uploading_csv('\General','\X_test20.csv')
y_train= uploading_csv('\General','\y_train3.csv')
y_test= uploading_csv('\General','\y_test20.csv')

X_test.keys()

Index(['WS1', 'WS3', 'WS4', 'WSHor', 'WDHor', 'WSVer', 'WDVer', 'T1', 'RH1',
       'T2', 'RH2', 'PR1', 'AD1', 'PR2', 'AD2', 'Rain', 'WD1', 'WD3', 'WD4',
       'TI', 'WSH', 'WD_bin', 'tod', 'WVeer'],
      dtype='object')

In [33]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.457 m/s as root mean
Wind MAE:  0.338 m/s in avg
Wind MAPE:  4.256 %
Power RMSE:  259.989 kW as root mean
Power MAE:  168.729 kW in avg
Power MAPE:  10.304 %

Modelling errors for test set:
Wind RMSE:  0.456 m/s as root mean
Wind MAE:  0.33 m/s in avg
Wind MAPE:  4.221 %
Power RMSE:  265.775 kW as root mean
Power MAE:  174.27 kW in avg
Power MAPE:  10.093 %

Showing the results of the modelling: 
NN results performed


### WTG43

In [34]:
#upload the dataset with file_folder, file_name
X_train= uploading_csv('\General','\X_train3.csv')
X_test= uploading_csv('\General','\X_test43.csv')
y_train= uploading_csv('\General','\y_train3.csv')
y_test= uploading_csv('\General','\y_test43.csv')

X_test.keys()

Index(['WS1', 'WS3', 'WS4', 'WSHor', 'WDHor', 'WSVer', 'WDVer', 'T1', 'RH1',
       'T2', 'RH2', 'PR1', 'AD1', 'PR2', 'AD2', 'Rain', 'WD1', 'WD3', 'WD4',
       'TI', 'WSH', 'WD_bin', 'tod', 'WVeer'],
      dtype='object')

In [35]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.457 m/s as root mean
Wind MAE:  0.338 m/s in avg
Wind MAPE:  4.256 %
Power RMSE:  259.989 kW as root mean
Power MAE:  168.729 kW in avg
Power MAPE:  10.304 %

Modelling errors for test set:
Wind RMSE:  0.557 m/s as root mean
Wind MAE:  0.43 m/s in avg
Wind MAPE:  4.965 %
Power RMSE:  295.341 kW as root mean
Power MAE:  185.425 kW in avg
Power MAPE:  10.188 %

Showing the results of the modelling: 
NN results performed


### WTG46

In [36]:
#upload the dataset with file_folder, file_name
X_train= uploading_csv('\General','\X_train3.csv')
X_test= uploading_csv('\General','\X_test46.csv')
y_train= uploading_csv('\General','\y_train3.csv')
y_test= uploading_csv('\General','\y_test46.csv')

X_test.keys()

Index(['WS1', 'WS3', 'WS4', 'WSHor', 'WDHor', 'WSVer', 'WDVer', 'T1', 'RH1',
       'T2', 'RH2', 'PR1', 'AD1', 'PR2', 'AD2', 'Rain', 'WD1', 'WD3', 'WD4',
       'TI', 'WSH', 'WD_bin', 'tod', 'WVeer'],
      dtype='object')

In [37]:
model_testing (X_train, X_test, y_train, y_test, PC, model, plot_error=False)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
power curve computation performed
power curve computation performed
Modelling errors for training set:
Wind RMSE:  0.457 m/s as root mean
Wind MAE:  0.338 m/s in avg
Wind MAPE:  4.256 %
Power RMSE:  259.989 kW as root mean
Power MAE:  168.729 kW in avg
Power MAPE:  10.304 %

Modelling errors for test set:
Wind RMSE:  0.416 m/s as root mean
Wind MAE:  0.308 m/s in avg
Wind MAPE:  4.202 %
Power RMSE:  252.933 kW as root mean
Power MAE:  162.5 kW in avg
Power MAPE:  11.253 %

Showing the results of the modelling: 
NN results performed
